In [ ]:
%pip install openai==1.58.1
%pip install python-dotenv==1.0.1
%pip install azure-search-documents==11.5.2

Load environment variables

In [9]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Azure Search
endpoint = os.getenv("ENDPOINT")
key = os.getenv("KEY")

# OpenAI
endpoint_openai = os.getenv("OPENAI_ENDPOINT")
openai_key = os.getenv("OPENAI_API_KEY")
chat_deployment_name = os.getenv("CHAT_DEPLOYMENT_NAME")

Create the clients

In [5]:
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

openai_client = AzureOpenAI(api_key=openai_key, azure_endpoint=endpoint_openai, api_version="2024-08-01-preview")

search_client = SearchClient(
    endpoint=endpoint, 
    index_name="hotels", 
    credential=AzureKeyCredential(key)
)

In [13]:
json_structure = [
    {
      "name": "hotelId",
      "type": "Edm.String",
      "key": "true",
      "retrievable": "true",
      "stored": "true",
      "searchable": "false",
      "filterable": "false",
      "sortable": "false",
      "facetable": "false",
      "synonymMaps": []
    },
    {
      "name": "hotelName",
      "type": "Edm.String",
      "key": "false",
      "retrievable": "true",
      "stored": "true",
      "searchable": "true",
      "filterable": "true",
      "sortable": "true",
      "facetable": "true",
      "synonymMaps": []
    },
    {
      "name": "description",
      "type": "Edm.String",
      "key": "false",
      "retrievable": "true",
      "stored": "true",
      "searchable": "true",
      "filterable": "true",
      "sortable": "true",
      "facetable": "true",
      "synonymMaps": []
    },
    {
      "name": "category",
      "type": "Edm.String",
      "key": "false",
      "retrievable": "true",
      "stored": "true",
      "searchable": "true",
      "filterable": "true",
      "sortable": "true",
      "facetable": "true",
      "synonymMaps": []
    },
    {
      "name": "rating",
      "type": "Edm.Int32",
      "key": "false",
      "retrievable": "true",
      "stored": "true",
      "searchable": "false",
      "filterable": "true",
      "sortable": "true",
      "facetable": "true",
      "synonymMaps": []
    }    
]

chat_history = [
        {
            "role": "system", 
            "content": f"""You are a helpful assistant that provides OData query, 
                          this is the structure of the index inside AzureAI Search in JSON {json_structure}.  
                          You just provide the OData query to execute the search, nothing else"""
        },
        {
            "role": "user",
            "content": "I want the list of all the luxury hotels"
        },
        {
            "role": "assistant",
            "content": "$filter=category eq 'Luxury'"
        }
]



In [18]:
from azure.core.rest import HttpRequest

chat_history.append({
    "role": "user",
    "content": "I want the list of all the hotels with a 5 ratings"
})

completion = openai_client.chat.completions.create(
    model=chat_deployment_name,
    messages=chat_history
)

print(completion.choices[0].message.content)

request = HttpRequest(method="GET", url=f"/docs/$filter=rating eq 5?api-version=2024-05-01-preview")
response = search_client.send_request(request)
response.raise_for_status()
response_body = response.json()
print(response_body)

$filter=rating eq 5


HttpResponseError: (InvalidName) The request is invalid. Details: query : Invalid document key: '$filter=rating eq 5'. Keys can only contain letters, digits, underscore (_), dash (-), or equal sign (=).
Code: InvalidName
Message: The request is invalid. Details: query : Invalid document key: '$filter=rating eq 5'. Keys can only contain letters, digits, underscore (_), dash (-), or equal sign (=).
Exception Details:	(InvalidDocumentKey) Invalid document key: '$filter=rating eq 5'. Keys can only contain letters, digits, underscore (_), dash (-), or equal sign (=). Parameters: query
	Code: InvalidDocumentKey
	Message: Invalid document key: '$filter=rating eq 5'. Keys can only contain letters, digits, underscore (_), dash (-), or equal sign (=). Parameters: query